In [35]:
import pixiedust

In [36]:
df = pixiedust.sampleData("https://raw.githubusercontent.com/veniceyong/cs658/master/cs658_test.csv",forcePandas=True)

Downloaded 221 bytes
Creating pandas DataFrame for 'https://raw.githubusercontent.com/veniceyong/cs658/master/cs658_test.csv'. Please wait...
Loading file using 'pandas'
Successfully created pandas DataFrame for 'https://raw.githubusercontent.com/veniceyong/cs658/master/cs658_test.csv'


In [ ]:
display(df)
from pyspark.sql import SQLContext
#print sc
#sqlCtx = SQLContext(sc)
sqlCtx.createDataFrame(df).show()
spdf=sqlCtx.createDataFrame(df)

SquareFeet,Bedrooms,Color,Price
2100,3,White,100000
2300,4,White,125000
2500,4,Brown,150000
3000,4,Brown,160000
2000,3,Brown,98000
2700,4,White,155000
3200,5,Brown,180000
3500,5,White,200000
2600,4,Brown,155000


In [38]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [39]:
assembler = VectorAssembler(inputCols=['SquareFeet','Bedrooms'],outputCol="features")
lr = LinearRegression(labelCol='Price', featuresCol='features')
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(spdf)

In [40]:
def get_prediction(square_feet, num_bedrooms):
    request_df = spark.createDataFrame([(square_feet, num_bedrooms)], ['SquareFeet','Bedrooms'])
    response_df = model.transform(request_df)
    return response_df

In [41]:
response = get_prediction(2400, 4)
response.show()

+----------+--------+------------+------------------+
|SquareFeet|Bedrooms|    features|        prediction|
+----------+--------+------------+------------------+
|      2400|       4|[2400.0,4.0]|135923.07692307708|
+----------+--------+------------+------------------+



In [45]:
# If you rerun this cell, save model and training data with new name
model.write.overwrite().save( "Home Prices Model" )
spdf.write.overwrite().save( "training-data.parquet" )

AttributeError: 'function' object has no attribute 'overwrite'

In [46]:
import json
import requests
import urllib3

In [47]:
!pip install watson-machine-learning-client

  Using cached https://files.pythonhosted.org/packages/b6/47/32daed1eb2169e93108480d7524fffa533f2375d8729d597c4b50cc2a6e8/watson_machine_learning_client-1.0.376-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/f2/764a5d530cf143ded9bc95216edb6e258c6554511e78de7c250557e8f3ed/tqdm-4.37.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e0/da/55f51ea951e1b7c63a579c09dd7db825bb730ec1fe9c0180fc77bfb31448/urllib3-1.25.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/b1/02eebed49c754b01b17de7705caa8c4ceecfb4f926cdafc220c863584360/lomond-0.3.3-py2.py3-none-any.whl
  Using cached https://fi

In [48]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials = { "apikey" : "v-Qtqjn4oA0H68Hu2gAn4lDgVakcnHcReYo0VPEO6i5w", 
                     "url" : "https://us-south.ml.cloud.ibm.com",
                     "instance_id" : "79c75949-2e56-465b-8837-cfb2f1339571" 
                  }

client = WatsonMachineLearningAPIClient( wml_credentials )

In [49]:
{
  "apikey": "v-Qtqjn4oA0H68Hu2gAn4lDgVakcnHcReYo0VPEO6i5w",
  "iam_apikey_description": "Auto-generated for key 6a875e5f-cc4e-424d-be85-6470a72cb074",
  "iam_apikey_name": "服务凭证-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/f0a88fe48d0a405f9bb65482d9ad83e6::serviceid:ServiceId-e221452c-02f6-468b-b609-2f2fca6b5744",
  "instance_id": "79c75949-2e56-465b-8837-cfb2f1339571",
  "url": "https://us-south.ml.cloud.ibm.com"
}

{'apikey': 'v-Qtqjn4oA0H68Hu2gAn4lDgVakcnHcReYo0VPEO6i5w',
 'iam_apikey_description': 'Auto-generated for key 6a875e5f-cc4e-424d-be85-6470a72cb074',
 'iam_apikey_name': '服务凭证-1',
 'iam_role_crn': 'crn:v1:bluemix:public:iam::::serviceRole:Writer',
 'iam_serviceid_crn': 'crn:v1:bluemix:public:iam-identity::a/f0a88fe48d0a405f9bb65482d9ad83e6::serviceid:ServiceId-e221452c-02f6-468b-b609-2f2fca6b5744',
 'instance_id': '79c75949-2e56-465b-8837-cfb2f1339571',
 'url': 'https://us-south.ml.cloud.ibm.com'}

In [50]:
from pyspark.ml import PipelineModel
pipeline_model = PipelineModel.load( "Home Prices Model" )
pipeline = Pipeline( stages = pipeline_model.stages )
train = spark.read.load( "training-data.parquet" )

In [51]:
# Store the PipelineModel in the Watson Machine Learning repository
model_details = client.repository.store_model( pipeline_model, 'Home Prices model', training_data=train, pipeline=pipeline )

In [52]:
# Deploy the stored model as an online web service deployment
model_id = model_details["metadata"]["guid"]
deployment_details = client.deployments.create( artifact_uid=model_id, name="Spark MLlib model deployment" )



#######################################################################################

Synchronous deployment creation for uid: '856f8eee-0676-4e79-851c-3e8e96ee1dbb' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='317726e6-cd3d-41e6-b62f-6c095e2db983'
------------------------------------------------------------------------------------------------




In [53]:
#### Test the deployment
model_endpoint_url = client.deployments.get_scoring_url( deployment_details )
payload = {'fields': ['SquareFeet','Bedrooms'], 'values': [[2400, 4]]}
client.deployments.score( model_endpoint_url, payload )

{'fields': ['SquareFeet', 'Bedrooms', 'features', 'prediction'],
 'values': [[2400, 4, [2400.0, 4.0], 137499.99999999968]]}

In [54]:
#### Testing the model locally gets the same results (testing again similar as above in cell "Test the Model witl Spark ML")
test_df = spark.createDataFrame([(2400, 4)], ['SquareFeet','Bedrooms'])
response_df = model.transform(test_df).show()

+----------+--------+------------+------------------+
|SquareFeet|Bedrooms|    features|        prediction|
+----------+--------+------------+------------------+
|      2400|       4|[2400.0,4.0]|135923.07692307708|
+----------+--------+------------+------------------+

